In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams["font.size"] = 16

from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.dummy import DummyClassifier

In [13]:
mushroom_traindf = pd.read_csv("train.csv")
mushroom_testdf = pd.read_csv("test.csv")

In [11]:
mushroom_traindf.count()

id                      3116945
class                   3116945
cap-diameter            3116941
cap-shape               3116905
cap-surface             2445922
cap-color               3116933
does-bruise-or-bleed    3116937
gill-attachment         2593009
gill-spacing            1858510
gill-color              3116888
stem-height             3116945
stem-width              3116945
stem-root                359922
stem-surface            1136084
stem-color              3116907
veil-type                159452
veil-color               375998
has-ring                3116921
ring-type               2988065
spore-print-color        267263
habitat                 3116900
season                  3116945
dtype: int64

In [21]:
mushroom_traindf.dtypes

id                        int64
class                    object
cap-diameter            float64
cap-shape                object
cap-surface              object
cap-color                object
does-bruise-or-bleed     object
gill-attachment          object
gill-spacing             object
gill-color               object
stem-height             float64
stem-width              float64
stem-root                object
stem-surface             object
stem-color               object
veil-type                object
veil-color               object
has-ring                 object
ring-type                object
spore-print-color        object
habitat                  object
season                   object
dtype: object

class: binary\
cap-diameter: numeric\
cap-shape: categorical\
cap-surface: categorical\
cap-color: categorical\
does-bruise-or-bleed: binary\
gill-attachment: categorical\
gill-spacing: categorical\
gill-color: categorical\
stem-height: numeric\
stem-width: numeric\
stem-root: categorical\
stem-surface: categorical\
stem-color: categorical\
veil-type: categorical\
veil-color: categorical\
has-ring: binary\
ring-type: categorical\
spore-print-color: categorical\
habitat: categorical\
season: categorical